# Fine-Tuning VGG16 Model

This notebook aims to finetune a pretrained VGG16 model by training it further on our specific dataset. The goal is to experiment with various training techniques and hyperparameters to achieve at least 95% performance on the test set. The dataset used consists of images categorized into three classes: pizza, hamburger, and hotdog. We will perform model training, validation, and testing to evaluate and improve the model's performance.

In [7]:
# PyTorch imports   
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision.models import VGG16_Weights

# Torchvision imports
from torchvision.datasets import ImageFolder

# Project-specific imports
from hamburger_hotdog_pizza_classifier.config import PROCESSED_DATA_DIR, MODELS_DIR, BATCH_SIZE, LEARNING_RATE, MOMENTUM, WEIGHT_DECAY
from utils import train_validate_model, test_model

# Model path definition
pretrained_model_path = MODELS_DIR / "vgg16.pth"
model_save_path = MODELS_DIR / 'fine_tuned_vgg16.pth'

# Load hyperparameters
batch_size = BATCH_SIZE
lr = LEARNING_RATE
momentum = MOMENTUM
weight_decay = WEIGHT_DECAY

print('Imported necessary libraries/variables')

Imported necessary libraries/variables


In [2]:
# Model loading and evaluation setup
model = torch.load(pretrained_model_path)
model.eval()

# Directory paths
image_path = PROCESSED_DATA_DIR / "pizza_hamburger_hotdog_100_percent"
train_dir = image_path / 'train'
test_dir = image_path / 'test'
valid_dir = image_path / 'valid'

# Data loading parameters
weights = VGG16_Weights.DEFAULT
transform = weights.transforms()

# Data preparation
train_data = ImageFolder(train_dir, transform=transform)
valid_data = ImageFolder(valid_dir, transform=transform)
test_data = ImageFolder(test_dir, transform=transform)

# Data loaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Loss function and optimizer setup
criterion = nn.CrossEntropyLoss()
exp_1_optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum) # Optimizer for initial training
exp_2_optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay) # Optimizer with weight decay

# Device setup
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Fine tuning 

In [19]:
train_validate_model(num_epochs=4, model=model, train_loader=train_loader, valid_loader=valid_loader, criterion=criterion, optimizer=exp_1_optimizer, device=device, model_save_path=model_save_path)

Overall Training Progress: 100%|██████████| 4/4 [01:04<00:00, 16.05s/it, Best Val Accuracy=93.11%, Current Train Accuracy=93.70%, Current Val Accuracy=92.00%]


Let's try adding a regularization term, specifically weight decay, to try to improve our model's performance on the validation set and avoid overfitting.

In [21]:
train_validate_model(num_epochs=10, model=model, train_loader=train_loader, valid_loader=valid_loader, criterion=criterion, optimizer=exp_2_optimizer, device=device, model_save_path=model_save_path)

Overall Training Progress: 100%|██████████| 10/10 [02:37<00:00, 15.72s/it, Best Val Accuracy=93.44%, Current Train Accuracy=97.93%, Current Val Accuracy=90.89%]


# Testing

Testing the model on the test set, we achieve 95% accuracy:

This model is renamed as "final_model.pth" in the `./models/` folder

In [3]:
fine_tuned_model_path = MODELS_DIR / "final_model.pth"
model = torch.load(fine_tuned_model_path)
test_model(model=model, test_loader=test_loader, device=device)

Test Accuracy: 95.07%
